In [1]:
# Imports
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os

# Check device
device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------


In [3]:

# -------------------------
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [4]:
# -------------------------
# Transform
# -------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)



In [5]:
# -------------------------
# Label Dictionaries
# -------------------------


label_dict = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}



In [6]:
disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_2",
    "chest": "C:\\Users\\sangeetha\\modelchest_19"
}

disease_labels = {
    
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma","Pituitary","Meningioma","No Tumor", "Other"],
    "bone": ["Osteoporosis","Normal","Osteopenia"],
    "alzheimer_mri": ["Non Demented","Mild Dementia","Very mild Dementia","Moderate Dementia"],
    "chest": ["TUBERCULOSIS","NORMAL","PNEUMONIA"]    
}



In [9]:
# -------------------------
# Predict Body Part
# -------------------------
def predict_body_part(image_path):
    body_part_model_path = "C:\\Users\\sangeetha\\class_2"  # Update as needed
    num_classes = len(label_dict)
    
    model = MyModel(num_classes=num_classes).to(device)
    model.load_state_dict(torch.load(body_part_model_path, map_location=device))
    model.eval()

    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return label_dict[predicted.item()]

  
# -------------------------
# Predict Disease
# -------------------------
def predict_disease(image_path, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    num_classes = len(labels)

    model = MyModel(num_classes=num_classes).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    image = preprocess_image(image_path)
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return labels[predicted.item()]



In [10]:
# -------------------------
# Full Pipeline
# -------------------------
def classify_image(image_path):
    print(f"\nProcessing Image: {image_path}")
    body_part = predict_body_part(image_path)
    print(f"Predicted Body Part: {body_part}")

    disease = predict_disease(image_path, body_part)
    print(f"Predicted Disease: {disease}")
    return body_part, disease



In [11]:
# -------------------------
# Example Usage
# -------------------------
if __name__ == "__main__":
    test_image_path = "C:\\Users\\sangeetha\\1\\Bone_Fracture_Binary_Classification\\dataset\\Bone_Fracture_Binary_Classification\\val\\not fractured\\1-rotated3.jpg"
    classify_image(test_image_path)



Processing Image: C:\Users\sangeetha\1\Bone_Fracture_Binary_Classification\dataset\Bone_Fracture_Binary_Classification\val\not fractured\1-rotated3.jpg
Predicted Body Part: Bone_Fracture_Binary_Classification
Predicted Disease: not fractured


C:\Users\sangeetha\AppData\Local\Temp\ipykernel_16140\1549561884.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(body_part_model_path, m

In [12]:
# -------------------------
# Example Usage
# -------------------------
if __name__ == "__main__":
    test_image_path = "C:\\Users\\sangeetha\\1\\chest\\train\\TUBERCULOSIS\\266.png"
    classify_image(test_image_path)



Processing Image: C:\Users\sangeetha\1\chest\train\TUBERCULOSIS\266.png


C:\Users\sangeetha\AppData\Local\Temp\ipykernel_16140\1549561884.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(body_part_model_path, m

Predicted Body Part: chest


C:\Users\sangeetha\AppData\Local\Temp\ipykernel_16140\1549561884.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_locati

Predicted Disease: TUBERCULOSIS


In [15]:
%%writefile app.py



import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Label dictionaries
label_dict = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}

disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_2",
    "chest": "C:\\Users\\sangeetha\\modelchest_19"
}

disease_labels = {
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma", "Pituitary", "Meningioma", "No Tumor", "Other"],
    "bone": ["Osteoporosis", "Normal", "Osteopenia"],
    "alzheimer_mri": ["Non Demented", "Mild Dementia", "Very mild Dementia", "Moderate Dementia"],
    "chest": ["TUBERCULOSIS", "NORMAL", "PNEUMONIA"]
}

# Preprocess image
def preprocess_image(image_file):
    image = Image.open(image_file).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Predict body part
def predict_body_part(image_tensor):
    model = MyModel(num_classes=len(label_dict)).to(device)
    model_path = "C:\\Users\\sangeetha\\class_2"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return label_dict[predicted.item()]

# Predict disease
def predict_disease(image_tensor, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    model = MyModel(num_classes=len(labels)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]

# Streamlit app
st.set_page_config(page_title="X-ray Classifier", layout="centered")
st.title("🧠🩻 AI Medical Image Classifier")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    with st.spinner("Classifying..."):
        image_tensor = preprocess_image(uploaded_file)
        body_part = predict_body_part(image_tensor)
        disease = predict_disease(image_tensor, body_part)
    st.success(f"**Predicted Body Part:** {body_part}")
    st.success(f"**Predicted Disease:** {disease}")


Writing app.py


In [16]:
!streamlit run app.py


^C


In [1]:
%%writefile app.py

import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model architecture
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Label dictionaries
label_dict = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}

disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_2",
    "chest": "C:\\Users\\sangeetha\\modelchest_19"
}

disease_labels = {
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma", "Pituitary", "Meningioma", "No Tumor", "Other"],
    "bone": ["Osteoporosis", "Normal", "Osteopenia"],
    "alzheimer_mri": ["Non Demented", "Mild Dementia", "Very mild Dementia", "Moderate Dementia"],
    "chest": ["TUBERCULOSIS", "NORMAL", "PNEUMONIA"]
}

# Path for scan type classifier
scan_model_path = "C:\\Users\\sangeetha\\modelmain_3"
scan_labels = ["xrayctscan", "others"]

# Preprocess image
def preprocess_image(image_file):
    image = Image.open(image_file).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Check if image is X-ray or CT scan
def is_medical_scan(image_tensor):
    model = MyModel(num_classes=2).to(device)
    model.load_state_dict(torch.load(scan_model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return scan_labels[predicted.item()] == "scan"

# Predict body part
def predict_body_part(image_tensor):
    model = MyModel(num_classes=len(label_dict)).to(device)
    model_path = "C:\\Users\\sangeetha\\class_2"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return label_dict[predicted.item()]

# Predict disease
def predict_disease(image_tensor, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    model = MyModel(num_classes=len(labels)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]

# Streamlit UI
st.set_page_config(page_title="X-ray Classifier", layout="centered")
st.title("🧠🩻 AI Medical Image Classifier")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    with st.spinner("Checking image type..."):
        image_tensor = preprocess_image(uploaded_file)
        if not is_medical_scan(image_tensor):
            st.error("❌ Please upload a valid X-ray or CT scan image.")
        else:
            with st.spinner("Classifying..."):
                body_part = predict_body_part(image_tensor)
                disease = predict_disease(image_tensor, body_part)
            st.success(f"**Predicted Body Part:** {body_part}")
            st.success(f"**Predicted Disease:** {disease}")


Overwriting app.py


In [2]:
!streamlit run app.py

^C


In [11]:
%%writefile app.py

import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model Definition
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transformations for preprocessing the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Label dictionaries and model paths for body parts and diseases
label_dict = {
    0: "xrayctscan",
    1: "others"
}

# Label dictionaries
label_dict1 = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}

disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_4",
    "chest": "C:\\Users\\sangeetha\\modelchest_19",
   
}

disease_labels = {
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma", "Pituitary", "Meningioma", "No Tumor", "Other"],
    "bone": ["Osteoporosis", "Normal", "Osteopenia"],
    "alzheimer_mri": ["Non Demented", "Mild Dementia", "Very mild Dementia", "Moderate Dementia"],
    "chest": ["TUBERCULOSIS", "NORMAL", "PNEUMONIA"],
    
}

# Preprocess image
def preprocess_image(image_file):
    image = Image.open(image_file).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Check if the image is X-ray or CT Scan
def is_xray_or_ctscan(image_tensor):
    # Load the model for X-ray vs CT Scan classification (dummy model for demonstration)
    model = MyModel(num_classes=2).to(device)  # 2 classes: X-ray or CT Scan, Other
    model_path = "C:\\Users\\sangeetha\\modelmain_3"  # Replace with actual model path
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        
    return predicted.item()  # 0 for X-ray/CT Scan, 1 for Other

# Predict body part
def predict_body_part(image_tensor):
    model = MyModel(num_classes=5).to(device)
    model_path = "C:\\Users\\sangeetha\\class_2"  # Replace with actual model path
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    
    return label_dict1[predicted.item()]

# Predict disease
def predict_disease(image_tensor, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    model = MyModel(num_classes=len(labels)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    
    return labels[predicted.item()]

# Streamlit app interface
st.set_page_config(page_title="X-ray or CT Scan Classifier", layout="centered")
st.title("🧠🩻 AI Medical Image Classifier")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    
    with st.spinner("Classifying..."):
        # Preprocess the image
        image_tensor = preprocess_image(uploaded_file)
        
        # Step 1: Check if it's X-ray or CT Scan or Other
        result = is_xray_or_ctscan(image_tensor)
        
        if result == 0:
            st.success("The image is identified as a valid X-ray or CT scan.")
            
            # Step 2: Classify Body Part and Predict Disease
            body_part = predict_body_part(image_tensor)
            disease = predict_disease(image_tensor, body_part)
            
            st.success(f"**Predicted Body Part:** {body_part}")
            st.success(f"**Predicted Disease:** {disease}")
        else:
            st.error("Please upload a valid X-ray or CT scan image. The uploaded image is classified as 'Other'.")
            st.stop()  # Stop further processing if it's not valid


Overwriting app.py


In [12]:
!streamlit run app.py

^C


In [30]:
%%writefile app.py

import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import base64
from io import BytesIO

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model Definition
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transformations for preprocessing the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Label dictionaries and model paths for body parts and diseases
label_dict = {
    0: "xrayctscan",
    1: "others"
}

label_dict1 = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}

disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_4",
    "chest": "C:\\Users\\sangeetha\\modelchest_19",
}

disease_labels = {
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma", "Pituitary", "Meningioma", "No Tumor", "Other"],
    "bone": ["Osteoporosis", "Normal", "Osteopenia"],
    "alzheimer_mri": ["Non Demented", "Mild Dementia", "Very mild Dementia", "Moderate Dementia"],
    "chest": ["TUBERCULOSIS", "NORMAL", "PNEUMONIA"],
}

# Preprocess image
def preprocess_image(image_file):
    image = Image.open(image_file).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Check if the image is X-ray or CT Scan
def is_xray_or_ctscan(image_tensor):
    model = MyModel(num_classes=2).to(device)
    model_path = "C:\\Users\\sangeetha\\modelmain_3"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

# Predict body part
def predict_body_part(image_tensor):
    model = MyModel(num_classes=5).to(device)
    model_path = "C:\\Users\\sangeetha\\class_2"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return label_dict1[predicted.item()]

# Predict disease
def predict_disease(image_tensor, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    model = MyModel(num_classes=len(labels)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]



# Streamlit app interface
st.set_page_config(page_title="Radiology report generator", layout="centered")
st.title("🧠Radiology report generator")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])



if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    
    with st.spinner("Classifying..."):
        image_tensor = preprocess_image(uploaded_file)
        result = is_xray_or_ctscan(image_tensor)
        
        if result == 0:
            st.success("The image is identified as a valid X-ray or CT scan.")

            body_part = predict_body_part(image_tensor)
            disease = predict_disease(image_tensor, body_part)

            st.success(f"**Predicted Body Part:** {body_part}")
            st.success(f"**Predicted Disease:** {disease}")

            
            # Diagnostic Report
            report = f"""
            📋 **Diagnostic Report**
            
            • Scan Type: X-ray / CT  
            • Predicted Body Part: {body_part}  
            • Predicted Disease: {disease}  

            
            ⚠️ *Please consult a medical professional for confirmation.*
            """


            st.markdown(report, unsafe_allow_html=True)

            # Create report text including base64 image for download
            download_report = f"""
            📋 **Diagnostic Report**
            
            • Scan Type: X-ray / CT  
            • Predicted Body Part: {body_part}  
            • Predicted Disease: {disease}  
            
            ⚠️ *Please consult a medical professional for confirmation.* """

            st.download_button("📥 Download Report", download_report, file_name="diagnostic_report.txt", mime="text/plain")

        else:
            st.error("Please upload a valid X-ray or CT scan image. The uploaded image is classified as 'Other'.")
            st.stop()

Overwriting app.py


In [ ]:
!streamlit run app.py

In [41]:
%%writefile app.py



import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import base64
from io import BytesIO

# Set page config at the top of the script
st.set_page_config(page_title="Radiology report generator", layout="centered")

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model Definition
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transformations for preprocessing the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Label dictionaries and model paths for body parts and diseases
label_dict = {
    0: "xrayctscan",
    1: "others"
}

label_dict1 = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}

disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_4",
    "chest": "C:\\Users\\sangeetha\\modelchest_19",
}

disease_labels = {
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma", "Pituitary", "Meningioma", "No Tumor", "Other"],
    "bone": ["Osteoporosis", "Normal", "Osteopenia"],
    "alzheimer_mri": ["Non Demented", "Mild Dementia", "Very mild Dementia", "Moderate Dementia"],
    "chest": ["TUBERCULOSIS", "NORMAL", "PNEUMONIA"],
}

# Preprocess image
def preprocess_image(image_file):
    image = Image.open(image_file).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Check if the image is X-ray or CT Scan
def is_xray_or_ctscan(image_tensor):
    model = MyModel(num_classes=2).to(device)
    model_path = "C:\\Users\\sangeetha\\modelmain_3"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

# Predict body part
def predict_body_part(image_tensor):
    model = MyModel(num_classes=5).to(device)
    model_path = "C:\\Users\\sangeetha\\class_2"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return label_dict1[predicted.item()]

# Predict disease
def predict_disease(image_tensor, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    model = MyModel(num_classes=len(labels)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]


# CSS styling for the application
st.markdown("""
    <style>
        body {
            background-color: #f7f7f7;
            font-family: 'Arial', sans-serif;
        }
        .css-1d391kg {
            background-color: #dfe7f3;
        }
        .stButton>button {
            background-color: #0066cc;
            color: white;
            font-weight: bold;
            border-radius: 10px;
            padding: 10px 20px;
            cursor: pointer;
        }
        .stButton>button:hover {
            background-color: #004c99;
        }
        h1 {
            color: #2c3e50;
            text-align: center;
        }
        .stFileUploader>label {
            font-size: 18px;
            color: #333;
        }
        .stSpinner>div {
            font-size: 16px;
            color: #666;
        }
        .stMarkdown {
            font-size: 16px;
            color: #333;
        }
    </style>
""", unsafe_allow_html=True)

# Streamlit app interface
st.title("🧠 Radiology report generator")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    
    with st.spinner("Classifying..."):
        image_tensor = preprocess_image(uploaded_file)
        result = is_xray_or_ctscan(image_tensor)
        
        if result == 0:
            st.success("The image is identified as a valid X-ray or CT scan.")

            body_part = predict_body_part(image_tensor)
            disease = predict_disease(image_tensor, body_part)

            st.success(f"**Predicted Body Part:** {body_part}")
            st.success(f"**Predicted Disease:** {disease}")

            # Diagnostic Report
            report = f"""
            📋 **Diagnostic Report**
            
            • Scan Type: X-ray / CT  
            • Predicted Body Part: {body_part}  
            • Predicted Disease: {disease}  

            ⚠️ *Please consult a medical professional for confirmation.*
            """

            st.markdown(report, unsafe_allow_html=True)

            # Create report text including base64 image for download
            download_report = f"""
            📋 **Diagnostic Report**
            
            • Scan Type: X-ray / CT  
            • Predicted Body Part: {body_part}  
            • Predicted Disease: {disease}  
            
            ⚠️ *Please consult a medical professional for confirmation.* """

            st.download_button("📥 Download Report", download_report, file_name="diagnostic_report.txt", mime="text/plain")

        else:
            st.error("Please upload a valid X-ray or CT scan image. The uploaded image is classified as 'Other'.")
            st.stop()


Overwriting app.py


In [42]:
!streamlit run app.py

^C


In [43]:
%%writefile app.py

import streamlit as st
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import base64
from io import BytesIO

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model Definition
class MyModel(nn.Module):
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=1, padding=0)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=4, stride=1, padding=0)
        self.bn4 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=3)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        self.fc1 = nn.Linear(6 * 6 * 128, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.pool2(x)
        x = self.relu(self.bn4(self.conv4(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Transformations for preprocessing the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Label dictionaries and model paths for body parts and diseases
label_dict = {
    0: "xrayctscan",
    1: "others"
}

label_dict1 = {
    0: "Bone_Fracture_Binary_Classification",
    1: "brain-tumor",
    2: "bone",
    3: "alzheimer_mri",
    4: "chest"
}

disease_model_paths = {
    "Bone_Fracture_Binary_Classification": "C:\\Users\\sangeetha\\bone_10",
    "brain-tumor": "C:\\Users\\sangeetha\\model_16",
    "bone": "C:\\Users\\sangeetha\\boneknee_20",
    "alzheimer_mri": "C:\\Users\\sangeetha\\modelalzhe_4",
    "chest": "C:\\Users\\sangeetha\\modelchest_19",
}

disease_labels = {
    "Bone_Fracture_Binary_Classification": ["fractured", "not fractured"],
    "brain-tumor": ["Glioma", "Pituitary", "Meningioma", "No Tumor", "Other"],
    "bone": ["Osteoporosis", "Normal", "Osteopenia"],
    "alzheimer_mri": ["Non Demented", "Mild Dementia", "Very mild Dementia", "Moderate Dementia"],
    "chest": ["TUBERCULOSIS", "NORMAL", "PNEUMONIA"],
}

# Preprocess image
def preprocess_image(image_file):
    image = Image.open(image_file).convert("RGB")
    image = transform(image).unsqueeze(0)
    return image.to(device)

# Check if the image is X-ray or CT Scan
def is_xray_or_ctscan(image_tensor):
    model = MyModel(num_classes=2).to(device)
    model_path = "C:\\Users\\sangeetha\\modelmain_3"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return predicted.item()

# Predict body part
def predict_body_part(image_tensor):
    model = MyModel(num_classes=5).to(device)
    model_path = "C:\\Users\\sangeetha\\class_2"
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return label_dict1[predicted.item()]

# Predict disease
def predict_disease(image_tensor, body_part):
    model_path = disease_model_paths[body_part]
    labels = disease_labels[body_part]
    model = MyModel(num_classes=len(labels)).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    return labels[predicted.item()]



# Streamlit app interface
st.set_page_config(page_title="Radiology report generator", layout="centered")
st.title("🧠Radiology report generator")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])



if uploaded_file is not None:
    st.image(uploaded_file, caption="Uploaded Image", use_column_width=True)
    
    with st.spinner("Classifying..."):
        image_tensor = preprocess_image(uploaded_file)
        result = is_xray_or_ctscan(image_tensor)
        
        if result == 0:
            st.success("The image is identified as a valid X-ray or CT scan.")

            body_part = predict_body_part(image_tensor)
            disease = predict_disease(image_tensor, body_part)

            st.success(f"**Predicted Body Part:** {body_part}")
            st.success(f"**Predicted Disease:** {disease}")

            
            # Diagnostic Report
            report = f"""
            📋 **Diagnostic Report**
            
            • Scan Type: X-ray / CT  
            • Predicted Body Part: {body_part}  
            • Predicted Disease: {disease}  

            
            ⚠️ *Please consult a medical professional for confirmation.*
            """


            st.markdown(report, unsafe_allow_html=True)

            # Create report text including base64 image for download
            download_report = f"""
            📋 **Diagnostic Report**
            
            • Scan Type: X-ray / CT  
            • Predicted Body Part: {body_part}  
            • Predicted Disease: {disease}  
            
            ⚠️ *Please consult a medical professional for confirmation.* """

            st.download_button("📥 Download Report", download_report, file_name="diagnostic_report.txt", mime="text/plain")

        else:
            st.error("Please upload a valid X-ray or CT scan image. The uploaded image is classified as 'Other'.")
            st.stop()

Overwriting app.py


In [ ]:
!streamlit run app.py